### IPMOF - InterPenetrating Metal Organic Frameworks

#### Initialize necessary information

1. Read FF_Parameters excel file to get force field parameters for atoms
2. Initialize force field selection, cut_off radius and grid size for energy map

#### Read structural information for MOF files in a given directory
1. Read MOF files in ".mol2" format from given directory and create a list
2. Create MOF objects for structure files
3. Initialize structural information for the MOFs

#### Read simulation input parameters
1. Read simulation parameters from input file

#### Calculate energy map
1. Determine packing amount of the MOF
2. Calculate packed coordinates for the base MOF
3. Calculate energy map

#### Start interpenetration
1. Energy map + mobile_mof

In [1]:
import os
import math
import yaml

input_dir = r'/home/kutay/Documents/git/IPMOF'
sim_par_path = os.path.join(input_dir, 'sim_par.yaml')
sim_dir_path = os.path.join(input_dir, 'sim_dir_linux.yaml')

# Read sim par yaml file
sim_par = yaml.load(open(sim_par_path, 'r'))
sim_dir = yaml.load(open(sim_dir_path, 'r'))

In [2]:
# Load interpenetration python libraries
os.chdir(sim_dir['python_lib_dir'])
from forcefield import read_ff_parameters
from crystal import *
from energymap import *
from visualize import *
from interpenetration import *
from geometry import Coor, Quaternion
from input import export_sim_par, export_sim_dir
%pylab inline

# Read excel file containing force field information
uff = read_ff_parameters(sim_dir['excel_file_path'], 'uff')

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


### File Input Options
- <b> Built-in read_mol2 </b>
<pre>
uc_size, uc_angle, atom_names, atom_coors = read_mol2(mol2_path)
mof = MOF()
mof.initialize(mof.mol2_path)
</pre> 
- <b> Ase pdb read (cif read gives error) </b>
<pre>
from ase.io import read
mof_atoms = read(mof_dir, format='pdb')
mof_atoms.get_positions()                # Coordinates
mof_atoms.get_chemical_symbols()         # Atom names
mof_atoms.get_cell()                     # Cell vectors? (check)
mof_obj.get_number_of_atoms()            # Num of atoms
mof_obj.get_volume()                     # Unit cell volume

</pre>
- <b> Open babel </b>
<pre>
babel -icif *.cif -opdb *.pdb
</pre>

In [3]:
# Create list of MOFs
mol2_list = get_mof_list(sim_dir['mol2_dir'], '.mol2')
print(mol2_list)

base_mof_index = 4
mobile_mof_index = 4

['QIGBIR.mol2', 'OFODAP.mol2', 'UNIGEE.mol2', 'ZIF90.mol2', 'SAHYIK.mol2', 'KINFAQ.mol2', 'NUVWIL.mol2']


In [4]:
# Read mol2 files and initialize MOF objects
base_mof = MOF()
base_mof.mol2_path = os.path.join(sim_dir['mol2_dir'], mol2_list[base_mof_index])
base_mof.initialize()
base_mof.initialize_ff(uff)
print('Base MOF selected as: ', base_mof.name)

mobile_mof = MOF()
mobile_mof.mol2_path = os.path.join(sim_dir['mol2_dir'], mol2_list[mobile_mof_index])
mobile_mof.initialize()
mobile_mof.initialize_ff(uff)
print('Mobile MOF selected as: ', mobile_mof.name)

Base MOF selected as:  SAHYIK
Mobile MOF selected as:  SAHYIK


In [5]:
cut_off = sim_par['cut_off']

base_mof.packing_factor = Packing.factor(base_mof.uc_size, cut_off)
uc_vectors = Packing.uc_vectors(base_mof.uc_size, base_mof.uc_angle)
trans_vec = Packing.translation_vectors(base_mof.packing_factor, uc_vectors)
base_mof.packed_coors = Packing.uc_coors(trans_vec, base_mof.packing_factor, uc_vectors, base_mof.atom_coors)
base_mof.edge_points = Packing.edge_points(uc_vectors)

print('Base MOF unit cell: ', base_mof.uc_size)
print('Packing factor:', base_mof.packing_factor)
print('Num of coor :', len(base_mof.packed_coors)*len(base_mof.packed_coors[0]))

Base MOF unit cell:  [25.669, 25.669, 25.669]
Packing factor: [3, 3, 3]
Num of coor : 11448


In [6]:
atom_list = get_uniq_atom_list([mobile_mof])
print('Calculating emap for', base_mof.name, 'with atoms:', atom_list['atom'])
emap = energy_map(base_mof, atom_list, cut_off, 1)

Calculating emap for SAHYIK with atoms: ['C', 'H', 'Zn', 'O']


In [21]:
# Run interpenetration
summary, new_structures = run_interpenetration(sim_par, base_mof, mobile_mof, emap, atom_list)

In [22]:
# Get minimum energy structure by sorting total structure energies
min_energy_structure = sorted(new_structures, key=lambda k: k['energy'])[0]

In [23]:
# Get structure information for the interpenetrating structure
ext_structure = save_extension(sim_par, base_mof, mobile_mof, emap, atom_list, min_energy_structure)

In [ ]:
# Join base and mobile layers

# Export to xyz format
export_xyz(joined_structure['atom_coors'], joined_structure['atom_names'], 'Joined', sim_dir['export_dir'])

In [34]:
export_xyz(ext_structure['atom_coors'], ext_structure['atom_names'], 'mobile_ext', sim_dir['export_dir'])

[3, 3, 3]


In [29]:
collision = check_extension(sim_par, base_mof, mobile_mof, emap, atom_list, min_energy_structure)

In [ ]:
def join_structures(base_mof, new_structure):
    joined_atom_names = []
    joined_atom_coors = []
    for atom, coor in zip(new_structure['atom_names'], new_structure['atom_coors']):
        joined_atom_names.append(atom)
        joined_atom_coors.append(coor)
        
    for atom, coor in zip(base_mof.atom_names, base_mof.atom_coors):
        joined_atom_names.append(atom)
        joined_atom_coors.append(coor)
        
    return joined_atom_names, joined_atom_coors

# Define file name and export directory
file_name = base_mof.name + '_' + mobile_mof.name
# export_dir = r'C:\Kutay\MOFs\IPMOF_Python\export'
export_dir = r'/home/kutay/Documents/Research/MOFs/IPMOF_Python/export'

# Join interpenetrating structures
joined_names, joined_coors = join_structures(base_mof, ext_structure)

# Export coordinates in .xyz format
export_xyz(joined_coors, joined_names, file_name, export_dir)

In [ ]:
packing_factor = Packing.factor(mobile_mof.uc_size, 50)
uc_vectors = Packing.uc_vectors(mobile_mof.uc_size, mobile_mof.uc_angle)
trans_vec = Packing.translation_vectors(packing_factor, uc_vectors)
packed_coors = Packing.uc_coors(trans_vec, packing_factor, uc_vectors, mobile_mof.atom_coors)

In [ ]:
packed_mof_path = os.path.join(export_dir, 'packed.xyz')
packed_mof_file = open(packed_mof_path, 'w')
packed_mof_file.write(str(len(packed_coors)*len(packed_coors[0])) + '\n')
packed_mof_file.write('Packed \n')

for unit_cell in packed_coors:
    for coor_index, coor in enumerate(unit_cell):
        atom_name = mobile_mof.atom_names[coor_index]
        packed_mof_file.write(atom_name + ' ' + str(coor[0]) + ' ' + str(coor[1]) + ' ' + str(coor[2]) + '\n')
        
packed_mof_file.close()

In [ ]:
def join_structures(base_structure, new_structure):
    joined_atom_names = []
    joined_atom_coors = []
    for atom, coor in zip(new_structure['atom_names'], new_structure['atom_coors']):
        joined_atom_names.append(atom)
        joined_atom_coors.append(coor)
        
    for atom, coor in zip(base_structure['atom_names'], base_structure['atom_coors']):
        joined_atom_names.append(atom)
        joined_atom_coors.append(coor)
        
    return joined_atom_names, joined_atom_coors

In [ ]:
packing_factor = Packing.factor(base_mof.uc_size, 20)
uc_vectors = Packing.uc_vectors(base_mof.uc_size, base_mof.uc_angle)
trans_vec = Packing.translation_vectors(packing_factor, uc_vectors)
packed_coors = Packing.uc_coors(trans_vec, packing_factor, uc_vectors, base_mof.atom_coors)

packed_names = []
for unit_cell in packed_coors:
    for coor_index, coor in enumerate(unit_cell):
        atom_name = mobile_mof.atom_names[coor_index]
        packed_names.append(atom_name)
        
packed_str = {'atom_names': packed_names, 'atom_coors': packed_coors}

In [ ]:
# Define file name and export directory
file_name = base_mof.name + '_' + mobile_mof.name
# export_dir = r'C:\Kutay\MOFs\IPMOF_Python\export'
export_dir = r'/home/kutay/Documents/Research/MOFs/IPMOF_Python/export'

# Join interpenetrating structures
joined_names, joined_coors = join_structures(packed_str, ext_structure)

# Export coordinates in .xyz format
export_xyz(joined_coors, joined_names, file_name, export_dir)